In [3]:
# !pip install beautifulsoup4 lxml

In [4]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
from PIL import Image, ImageTk
import requests
from io import BytesIO
from bs4 import BeautifulSoup

def open_file():
    filepath = filedialog.askopenfilename(filetypes=[("HTML files", "*.html;*.htm")])
    if not filepath:
        return
    with open(filepath, 'r', encoding='utf-8') as file:
        content = file.read()

    soup = BeautifulSoup(content, 'lxml')
    chapter_content = soup.find('div', id='chapter-content')

    if chapter_content:
        text_area.delete('1.0', tk.END)  # Clear current text area content
        for child in chapter_content.children:
            if child.name in ['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
                text_area.insert(tk.END, child.get_text(strip=True) + '\n\n')
            elif child.name == 'img':
                img_url = child.get('src')
                try:
                    if img_url.startswith('http://') or img_url.startswith('https://'):
                        response = requests.get(img_url)
                        img_data = Image.open(BytesIO(response.content))
                    else:
                        img_data = Image.open(img_url)  # Assume local path or malformed URL

                    img_data.thumbnail((300, 300))  # Resize image if needed
                    img = ImageTk.PhotoImage(img_data)
                    text_area.image_create(tk.END, image=img)
                    text_area.image = img  # Keep a reference!
                    text_area.insert(tk.END, '\n\n')  # Add space after the image
                except Exception as e:
                    text_area.insert(tk.END, f"Error loading image: {img_url}\nError: {e}\n\n")
    else:
        text_area.insert(tk.END, "No 'chapter-content' element found.")

def main():
    global text_area
    root = tk.Tk()
    root.title("HTML Content Viewer")
    root.geometry("600x500")

    open_button = tk.Button(root, text="Open HTML File", command=open_file)
    open_button.pack(expand=True)

    text_area = scrolledtext.ScrolledText(root, wrap=tk.WORD)
    text_area.pack(expand=True, fill=tk.BOTH)

    root.mainloop()

if __name__ == "__main__":
    main()